<a href="https://colab.research.google.com/github/HumphreyChou/CLIC-2021-Learned-Image-Compression/blob/main/CLIC2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CLIC2021

此笔记本是私人的，你可以在此处运行代码，但是所有结果不会被保存，
选择：文件 > 在云端硬盘中保存一份副本 可以创建你自己的笔记本

使用时一定选择 修改 > 笔记本设置 > 硬件加速器 GPU

## 环境配置
绑定云端硬盘
绑定自己的云端硬盘，可以使用自己上传的代码

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

切换到当前工作路径

In [ ]:
import os
os.chdir("/content/drive/My Drive/Colab Notebooks/CLIC2021")

### 安装配置 Anaconda

In [ ]:
!wget https://repo.anaconda.com/archive/Anaconda3-5.2.0-Linux-x86_64.sh
!bash Anaconda3-5.2.0-Linux-x86_64.sh -bfp /usr/local
!conda update -y -q
!conda env list

In [ ]:
!conda install -y -q python=3.7 cudatoolkit=10.0 cudnn

In [ ]:
import sys
sys.path.insert(0, "/usr/local/lib/python3.7/site-packages/")
!source /usr/local/etc/profile.d/conda.sh
!source ~/.bashrc

检查 python 版本，运行结果应该为 `Python 3.7.*`

In [ ]:
!python --version

### 上传代码
此处使用未被修改的 Google HiFiC 代码，以后可以上传自己修改后的代码



In [ ]:
!git clone https://github.com/tensorflow/compression

### 安装第三方包

In [ ]:
%%shell
cd compression/models/

pip install -r hific/requirements.txt


## 训练

In [ ]:
import os
os.chdir("compression/models")

先训练 MSE + LPIPS 模型

In [ ]:
%%shell
# num_steps 1M过大会占满硬盘，1K过小欠拟合，可通过修改源码改一个合适的值, 可选择 --no-image-summaries 减小硬盘消耗
python -m hific.train --config mselpips --ckpt_dir ckpts/mse_lpips \
    --num_steps 1k --tfds_dataset_name coco2014


再训练 GAN 模型

In [ ]:
%%shell
python -m hific.train --config hific --ckpt_dir ckpts/hific \
                --init_autoencoder_from_ckpt_dir ckpts/mse_lpips \
                --num_steps 1k \
                --tfds_dataset_name coco2014

## 评估

In [ ]:
%%shell
python -m hific.evaluate --config hific --ckpt_dir ckpts/hific --out_dir out/ \
                   --tfds_dataset_name coco2014

## 保存模型
训练产生的模型参数和 checkpoint 数据存储在训练模型时指定的 `ckpt_dir`，评估时只需要使用这其中的数据即可。因此可以指定 `ckpt_dir` 指向云端磁盘，可在虚拟机消失后仍保存训练模型

也可以把得到的数据移动到云端磁盘

In [ ]:
!mv ckpts /content/drive/My\ Drive/Colab\ Notebooks/CLIC2021

## 打扫战场
配置环境、训练过程中会占用硬盘空间，直到占满。colab运行在虚拟机上，且训练产生出的参数和 checkpoint 存储在 ckpt_dir（可以是你指定的云端磁盘位置），因此可以直接选择 代码执行程序 > 将该运行时恢复出厂设置，即可被分配新的虚拟机，和所有的烦恼说拜拜

实际上，被分配的虚拟机存活时间最长12小时，即12小时后一切配置都会重置，这是谷歌的动态分配策略，以保证免费资源供应